## Predicting Class for New Images using a Deep Learning Model

In this notebook, we will use the image pre-processing functions as well as the saved Deep Learning model in order to predict the class of new images. 

In [1]:
import numpy as np
from PIL import Image
from PIL import ImageOps
import sys
import os
import requests
from io import BytesIO

from keras.models import load_model

Using TensorFlow backend.


In [2]:
def resize(image_path):
    """
    Pick a basic color (Black) and pad the images that do not have a 1:1 aspect ratio.
    Reshape without stretching to a 128x128 pixel array shape
    """
    
    im = Image.open(image_path)
        
    desired_size = 128
    old_size = im.size  # old_size[0] is in (width, height) format

    ratio = float(desired_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])

    im = im.resize(new_size, Image.ANTIALIAS)
    
    # create a new image and paste the resized on it
    new_im = Image.new("RGB", (desired_size, desired_size))
    new_im.paste(im, ((desired_size-new_size[0])//2,
                        (desired_size-new_size[1])//2))

    delta_w = desired_size - new_size[0]
    delta_h = desired_size - new_size[1]
    padding = (delta_w//2, delta_h//2, delta_w-(delta_w//2), delta_h-(delta_h//2))
    new_im = ImageOps.expand(im, padding)

    filename, file_extension = os.path.splitext(image_path)
    new_filename = filename + "_resized.jpeg"
    new_im.save(new_filename, "JPEG")
    
    return new_filename

In [3]:
def equalize_image(image_path): #"imagename_resized.JPEG"
    """
    Ensure for each image that the pixel range is [0,255] (constrast stretching)
    by applying the equalise method (normalise works also)
    """
    im = Image.open(image_path)
    
    # Plotting histogram for resized image
    #im_array = np.array(im)
    #plt.hist(im_array.flatten(), bins=50, range=(0.0, 300))
    
    # Equalize image
    im_out = ImageOps.equalize(im)
    
    # Save equalized image
    filename, file_extension = os.path.splitext(image_path) 
    new_filename = filename + "_equalized.jpeg"
    im_out.save(new_filename, "JPEG")
    
    return new_filename

In [4]:
def image_to_array(image_path):
    """
    Input: Image path
    Output: Flatten array of 3x128x128 image pixels in range[0,255]
    """
    im = Image.open(image_path)
    return np.array(im, dtype=float)

In [5]:
def label_decoder(key):
    label_mapping = {
        "1" : "axes",
        "2" : "boots",
        "3" : "carabiners",
        "4" : "crampons",
        "5" : "gloves",
        "6" : "hardshell_jackets",
        "7" : "harnesses",
        "8" : "helmets",
        "9" : "insulated_jackets",
        "10" : "pulleys",
        "11" : "rope",
        "12" : "tents",
    }
    return label_mapping[key]

In [18]:
def run_model(data):
    """
    Load Keras model (.yaml) and weights (.h5), and predict image label
    Input: Image pixels array
    Output: Predicted label
    """
    
    # Load Keras Model
    loaded_model = load_model('gear_classifier.model')

    # Predict results array (array of 12 elements, one 1.0 and 11 are 0.0's)
    result_array = loaded_model.predict(data).tolist()[0]
    
    # Get the position of the element 1.0 within the array
    index = result_array.index(max(result_array)) + 1
    
    # Decode results
    predicted_label = label_decoder(str(index))
    
    return predicted_label

### Running the model on new image data

In [19]:
def main(image_path):
    # Pre-process image
    print('Image :', image_path.split('/')[1])
    resized_image_path = resize(image_path)
    preprocessed_img_path = equalize_image(resized_image_path)

    # Converting image to Numpy array
    numpy_data = image_to_array(preprocessed_img_path)
    numpy_data = numpy_data.reshape(1, 3, 128, 128)

    # Predict label using Keras Mutli-Class model
    result_label = run_model(numpy_data)

    print('Predicted : ' + result_label + '\n')

    # Cleaning up
    os.remove(resized_image_path)
    os.remove(preprocessed_img_path)

In [22]:
for filename in os.listdir("unseen_images"):
    filepath = 'new_test_images/' + filename
    main(filepath)

Image : boots.jpg
Predicted : boots

Image : carabiners.jpg
Predicted : carabiners

Image : crampons.jfif
Predicted : crampons

Image : gloves.jpg
Predicted : gloves

Image : harness.jpg
Predicted : harnesses

Image : helmet.jpg
Predicted : helmets

Image : jacket.jfif
Predicted : hardshell_jackets

Image : rope.jfif
Predicted : rope

Image : tent.jfif
Predicted : tents

